In [1]:
# train_pipe_mlp.py
import numpy as np, torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# ───────────────── CONFIG ──────────────────
NPZ_PATH   = "pipe_network_data.npz"
BATCH_SIZE = 64
EPOCHS     = 40
LR         = 3e-4
LAMBDA_Z   = 10.0          # weight for z-regression loss
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"
N_SLOTS    = 1365
F_IN       = 7
# ────────────────────────────────────────────


In [2]:
# ---------- Dataset wrapper -----------------
class PipeDataset(Dataset):
    def __init__(self, split):
        d = np.load(NPZ_PATH)
        self.X   = torch.tensor(d[f"{split}_features"],     dtype=torch.float32)
        self.y_p = torch.tensor(d[f"{split}_pipe_labels"], dtype=torch.long)
        self.y_z = torch.tensor(d[f"{split}_z_labels"],    dtype=torch.float32).unsqueeze(1)

    def __len__(self):              return len(self.y_p)
    def __getitem__(self, idx):     return self.X[idx], self.y_p[idx], self.y_z[idx]

In [3]:
# ---------- MLP with masking ---------------
class MaskedMLP(nn.Module):
    def __init__(self, in_slots=N_SLOTS, feat_dim=F_IN, hidden=2048, classes=N_SLOTS):
        super().__init__()
        flat_in = in_slots * feat_dim
        self.mask_index = feat_dim - 1          # mask is last feature (index 6)
        self.net = nn.Sequential(
            nn.Linear(flat_in, hidden),
            nn.BatchNorm1d(hidden),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden, hidden//2),
            nn.BatchNorm1d(hidden//2),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        self.pipe_head = nn.Linear(hidden//2, classes)
        self.z_head    = nn.Linear(hidden//2, 1)

    def forward(self, x):
        # x shape: [B, 1365, 7]
        # zero-out features for missing slots using the mask bit
        mask = x[:, :, self.mask_index: self.mask_index+1]   # [B, n, 1]
        x_use = x.clone()
        x_use[:, :, :-1] = x_use[:, :, :-1] * (1.0 - mask)   # keep mask channel itself
        x_flat = x_use.view(x.size(0), -1)
        h  = self.net(x_flat)
        return self.pipe_head(h), self.z_head(h)

In [4]:
# ---------- Training / validation ----------
def run_epoch(loader, model, opt=None):
    train_mode = opt is not None
    if train_mode:
        model.train()
    else:
        model.eval()

    loss_pipe_sum = loss_z_sum = 0.0
    correct = n = 0

    for X, y_p, y_z in loader:
        X, y_p, y_z = X.to(DEVICE), y_p.to(DEVICE), y_z.to(DEVICE)

        if train_mode:
            opt.zero_grad()

        p_logits, z_pred = model(X)
        loss_pipe = F.cross_entropy(p_logits, y_p)
        loss_z    = F.mse_loss(z_pred, y_z)
        loss      = loss_pipe + LAMBDA_Z * loss_z

        if train_mode:
            loss.backward()
            opt.step()

        loss_pipe_sum += loss_pipe.item() * X.size(0)
        loss_z_sum    += loss_z.item()    * X.size(0)
        preds          = p_logits.argmax(1)
        correct       += (preds == y_p).sum().item()
        n             += X.size(0)

    return (loss_pipe_sum/n, loss_z_sum/n, correct/n)

In [5]:
def main():
    train_ds = PipeDataset("train")
    val_ds   = PipeDataset("val")

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE)

    model = MaskedMLP().to(DEVICE)
    opt   = torch.optim.Adam(model.parameters(), lr=LR)

    for ep in range(1, EPOCHS+1):
        tr_lp, tr_lz, tr_acc = run_epoch(train_loader, model, opt)
        va_lp, va_lz, va_acc = run_epoch(val_loader,   model)

        print(f"Ep {ep:02d} | "
              f"Train CE {tr_lp:.4f}  MSE {tr_lz:.4f}  Acc {tr_acc:.3f} | "
              f"Val CE {va_lp:.4f}  MSE {va_lz:.4f}  Acc {va_acc:.3f}")

    torch.save(model.state_dict(), "pipe_mlp.pt")
    print("✔ saved pipe_mlp.pt")

In [7]:
if __name__ == "__main__":
    main()

ValueError: num_samples should be a positive integer value, but got num_samples=0